In [4]:
# this is to debus some weird values in the run_experiment function -- no need for others to see
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from src.util.modeling import calc_metrics_for_preds_df

In [5]:
results3 = pd.read_csv('results/linear_regression_detrended_demeaned_20230820-161104/metric_results.csv')
preds3 = pd.read_csv('results/linear_regression_detrended_demeaned_20230820-161104/ensemble_preds.csv')

In [6]:
results3

,mape,rmse,mae,r2,model
0,3.031269e+14,0.259437,0.067308,0.0,y_pred_1
1,3.031269e+14,0.259437,0.067308,0.0,y_pred_2
2,3.031269e+14,0.259437,0.067308,0.0,y_pred_3
3,3.464307e+14,0.277350,0.076923,0.0,y_pred_4
4,3.897346e+14,0.294174,0.086538,0.0,y_pred_5
5,4.330384e+14,0.310087,0.096154,0.0,y_pred_6
6,6.062538e+14,0.438529,0.134615,0.0,y_pred_7
7,5.629500e+14,0.427425,0.125000,0.0,y_pred_8
8,6.495576e+14,0.528059,0.144231,0.0,y_pred_9
9,3.031269e+14,0.259437,0.067308,0.0,ensemble_preds_2


In [7]:
calc_metrics_for_preds_df(preds3)

Calculating Metrics
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: y_true, dtype: float64
0    0.869982
1   -0.295499
2   -0.141847
3   -0.161531
4   -0.119130
Name: y_pred_1, dtype: float64
0.18935965237173524
{'mape': 852800059860353.5, 'rmse': 0.2553500772112193, 'mae': 0.18935965237173524, 'r2': 0.0}
Calculating Metrics
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: y_true, dtype: float64
0    0.918582
1   -0.314482
2   -0.469626
3   -0.455008
4   -0.055285
Name: y_pred_2, dtype: float64
0.18192389132059492
{'mape': 819312369161221.9, 'rmse': 0.2584089603706063, 'mae': 0.18192389132059492, 'r2': 0.0}
Calculating Metrics
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: y_true, dtype: float64
0    0.923600
1   -0.402845
2   -0.455416
3   -0.431391
4   -0.006174
Name: y_pred_3, dtype: float64
0.17944548705963573
{'mape': 808150628655092.6, 'rmse': 0.2578947579386667, 'mae': 0.17944548705963573, 'r2': 0.0}
Calculating Metrics
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name:

,mape,rmse,mae,r2,model
0,8.528001e+14,0.255350,0.189360,0.0,y_pred_1
1,8.193124e+14,0.258409,0.181924,0.0,y_pred_2
2,8.081506e+14,0.257895,0.179445,0.0,y_pred_3
3,8.001015e+14,0.262245,0.177658,0.0,y_pred_4
4,8.558662e+14,0.284475,0.190040,0.0,y_pred_5
5,8.825121e+14,0.295163,0.195957,0.0,y_pred_6
6,1.032166e+15,0.390801,0.229187,0.0,y_pred_7
7,1.049235e+15,0.399332,0.232977,0.0,y_pred_8
8,1.116195e+15,0.496137,0.247845,0.0,y_pred_9
9,8.307613e+14,0.254504,0.184466,0.0,ensemble_preds_2


In [77]:
res2 = calc_metrics_for_preds_df(preds2)

Calculating Metrics
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: y_true, dtype: float64
0    0.869982
1   -0.295499
2   -0.141847
3   -0.161531
4   -0.119130
Name: y_pred_1, dtype: float64
0.18935965237173524
{'mape': 852800059860353.5, 'rmse': 0.2553500772112193, 'mae': 0.18935965237173524, 'r2': 0.0}
Calculating Metrics
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: y_true, dtype: float64
0    0.918582
1   -0.314482
2   -0.469626
3   -0.455008
4   -0.055285
Name: y_pred_2, dtype: float64
0.18192389132059492
{'mape': 819312369161221.9, 'rmse': 0.2584089603706063, 'mae': 0.18192389132059492, 'r2': 0.0}
Calculating Metrics
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name: y_true, dtype: float64
0    0.923600
1   -0.402845
2   -0.455416
3   -0.431391
4   -0.006174
Name: y_pred_3, dtype: float64
0.17944548705963573
{'mape': 808150628655092.6, 'rmse': 0.2578947579386667, 'mae': 0.17944548705963573, 'r2': 0.0}
Calculating Metrics
0    0.0
1    0.0
2    0.0
3    0.0
4    0.0
Name:

In [7]:
results

,mape,rmse,mae,r2,model
0,3.031269e+14,0.259437,0.067308,0.0,y_pred_1
1,3.031269e+14,0.259437,0.067308,0.0,y_pred_2
2,3.031269e+14,0.259437,0.067308,0.0,y_pred_3
3,3.464307e+14,0.277350,0.076923,0.0,y_pred_4
4,3.897346e+14,0.294174,0.086538,0.0,y_pred_5
5,4.330384e+14,0.310087,0.096154,0.0,y_pred_6
6,6.062538e+14,0.438529,0.134615,0.0,y_pred_7
7,5.629500e+14,0.427425,0.125000,0.0,y_pred_8
8,6.495576e+14,0.528059,0.144231,0.0,y_pred_9
9,3.031269e+14,0.259437,0.067308,0.0,ensemble_preds_2


In [61]:
mean_absolute_error(preds['y_true'], preds['y_pred_1'])

0.18935965237173524

In [64]:
mean_squared_error(preds['y_true'], preds['ensemble_preds_9'])**.5

0.2910119420362771

In [32]:
r2_score(preds['y_true'], preds['ensemble_preds_6'])

0.0

In [36]:
np.sum((preds['y_true'].mean() - preds['y_true'])**2)

0.0